# Share Local Media - Direct Mail Analytics
## Phase 2: Analysis

In [ ]:
import pandas as pd
import plotly.express as px

# Load enriched data
df = pd.read_csv('Dataset/direct_mail_campaigns_enriched.csv')
print(df.head())

### Summary Tables: CPA/ROAS by Channel and Location

In [ ]:
# CPA/ROAS by Channel
channel_summary = df.groupby('Channel_Used')[['cpa', 'roas']].mean().reset_index()
print("Channel Summary:")
print(channel_summary)

# CPA/ROAS by Location
location_summary = df.groupby('Location')[['cpa', 'roas']].mean().reset_index()
print("\nLocation Summary:")
print(location_summary.head())

### Market Tiers

In [ ]:
# Bucket Locations by CPA percentile
# We calculate average CPA per location to classify the location itself
location_perf = df.groupby('Location')['cpa'].mean().reset_index()
cpa_33 = location_perf['cpa'].quantile(0.33)
cpa_66 = location_perf['cpa'].quantile(0.66)

def get_tier(cpa):
    if cpa <= cpa_33: return 'Top' # Lower CPA is better
    elif cpa <= cpa_66: return 'Middle'
    else: return 'Lower'

location_perf['Market_Tier'] = location_perf['cpa'].apply(get_tier)

# Merge back to main df so each campaign has its location's tier
df = df.merge(location_perf[['Location', 'Market_Tier']], on='Location')
print(df[['Location', 'Market_Tier']].drop_duplicates().head())

### Visualizations

In [ ]:
# 1. Bar: Avg CPA by Channel_Used
fig1 = px.bar(channel_summary, x='Channel_Used', y='cpa', title='Average CPA by Channel')
fig1.show()

# 2. Bar: Avg ROAS by Market Tier
tier_summary = df.groupby('Market_Tier')['roas'].mean().reset_index()
# Order tiers logically
tier_order = {'Top': 1, 'Middle': 2, 'Lower': 3}
tier_summary['order'] = tier_summary['Market_Tier'].map(tier_order)
tier_summary = tier_summary.sort_values('order')

fig2 = px.bar(tier_summary, x='Market_Tier', y='roas', title='Average ROAS by Market Tier')
fig2.show()

# 3. Scatter: Audience Size vs CPA
# FIX: Shuffle data and add transparency to avoid 'Wall of Green' overplotting
plot_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
fig3 = px.scatter(plot_df, x='audience_size_mailed', y='cpa', size='revenue', color='Channel_Used', 
                  title='Audience Size vs CPA', hover_data=['Location', 'Market_Tier'],
                  opacity=0.6)
fig3.show()